In [263]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
import random
%matplotlib inline
import seaborn as sns
from sklearn.metrics import accuracy_score



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

import datetime as dt

In [264]:
train = pd.read_csv("../input/flight-delay-prediction-challenge/Train (8).csv")
test = pd.read_csv("../input/flight-delay-prediction-challenge/Test (9).csv")
sample = pd.read_csv('../input/flight-delay-prediction-challenge/SampleSubmission (5).csv')

In [265]:
train

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0
...,...,...,...,...,...,...,...,...,...,...
107828,train_id_107828,2018-07-05,WKL 0000,TUN,TUN,2018-07-05 23:00:00,2018-07-06 02.00.00,SCH,TU 32AIML,0.0
107829,train_id_107829,2018-01-13,UG 0003,DJE,TUN,2018-01-13 08:00:00,2018-01-13 09.00.00,SCH,UG AT7AT7,0.0
107830,train_id_107830,2018-11-07,SGT 0000,TUN,TUN,2018-11-07 05:00:00,2018-11-07 12.50.00,SCH,TU 736IOK,0.0
107831,train_id_107831,2018-01-23,UG 0010,TUN,DJE,2018-01-23 18:00:00,2018-01-23 18.45.00,ATA,TU CR9ISA,0.0


In [266]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [267]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
sample = reduce_mem_usage(sample)

Memory usage after optimization is: 7.61 MB
Decreased by 7.5%
Memory usage after optimization is: 0.64 MB
Decreased by 0.0%
Memory usage after optimization is: 0.09 MB
Decreased by 37.5%


In [268]:
train.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC', 'target'],
      dtype='object')

In [269]:
test.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC'],
      dtype='object')

In [270]:
train.drop("ID", axis=1, inplace=True)
test.drop("ID", axis=1, inplace=True)

In [271]:
gc.collect()

147

In [272]:

train["DATOP"]=pd.to_datetime(train["DATOP"])
train['Date_year'] =train["DATOP"].dt.year
train['Date_month'] = train["DATOP"].dt.month
train['Date_day'] = train["DATOP"].dt.day
train.drop("DATOP", axis=1, inplace=True)

test["DATOP"]=pd.to_datetime(test["DATOP"])
test['Date_year'] =test["DATOP"].dt.year
test['Date_month'] = test["DATOP"].dt.month
test['Date_day'] = test["DATOP"].dt.day
test.drop("DATOP", axis=1, inplace=True)

In [273]:

train["STD"]=pd.to_datetime(train["STD"])
train['STD_Date_year'] =train["STD"].dt.year
train['STD_Date_month'] = train["STD"].dt.month
train['STD_Date_day'] = train["STD"].dt.day
train['STD_Date_hour'] = train["STD"].dt.hour
train['STD_Date_minute'] =train["STD"].dt.minute
train['STD_Date_second'] = train["STD"].dt.second

train.drop("STD", axis=1, inplace=True)

test["STD"]=pd.to_datetime(test["STD"])
test['STD_Date_year'] =test["STD"].dt.year
test['STD_Date_month'] = test["STD"].dt.month
test['STD_Date_day'] = test["STD"].dt.day
test['STD_Date_hour'] = test["STD"].dt.hour
test['STD_Date_minute'] =test["STD"].dt.minute
test['STD_Date_second'] = test["STD"].dt.second

test.drop("STD", axis=1, inplace=True)

In [274]:
train["STAA"]=pd.to_datetime(train["STA"], format='%Y-%m-%d %H.%M.%S')
train['STA_Date_year'] =train["STAA"].dt.year
train['STA_Date_month'] = train["STAA"].dt.month
train['STA_Date_day'] = train["STAA"].dt.day
train['STA_Date_hour'] = train["STAA"].dt.hour
train['STA_Date_minute'] =train["STAA"].dt.minute
train['STA_Date_second'] = train["STAA"].dt.second
train.drop("STA", axis=1, inplace=True)
train.drop("STAA", axis=1, inplace=True)

test["STAA"]=pd.to_datetime(test["STA"], format='%Y-%m-%d %H.%M.%S')
test['STA_Date_year'] =test["STAA"].dt.year
test['STA_Date_month'] = test["STAA"].dt.month
test['STA_Date_day'] = test["STAA"].dt.day
test['STA_Date_hour'] = test["STAA"].dt.hour
test['STA_Date_minute'] =test["STAA"].dt.minute
test['STA_Date_second'] = test["STAA"].dt.second
test.drop("STA", axis=1, inplace=True)
test.drop("STAA", axis=1, inplace=True)

In [275]:
# train_df = train.drop("target", axis=1)

In [276]:
from category_encoders import CountEncoder
enc = CountEncoder(normalize=True, cols=['DEPSTN', 'ARRSTN','STATUS','AC',"FLTID"])
train = enc.fit_transform(train)
test=enc.fit_transform(test)

In [277]:
train["mean"]=train.mean(axis=1)
train["count"]=train.count(axis=1)
train["std"]=train.std(axis=1)
train["min"]=train.min(axis=1)
train["max"]=train.max(axis=1)
train["kurt"]=train.kurt(axis=1)
train["skew"]=train.skew(axis=1)
train["q1"]=train.quantile(q=0.1,axis=1)
train["q2"]=train.quantile(q=0.2,axis=1)
train["q3"]=train.quantile(q=0.3,axis=1)
train["q4"]=train.quantile(q=0.4,axis=1)
train["q5"]=train.quantile(q=0.5,axis=1)
train["q6"]=train.quantile(q=0.6,axis=1)
train["q7"]=train.quantile(q=0.7,axis=1)
train["q8"]=train.quantile(q=0.8,axis=1)
train["q9"]=train.quantile(q=0.9,axis=1)

test["mean"]=test.mean(axis=1)
test["count"]=test.count(axis=1)
test["std"]=test.std(axis=1)
test["min"]=test.min(axis=1)
test["max"]=test.max(axis=1)
test["kurt"]=test.kurt(axis=1)
test["skew"]=test.skew(axis=1)
test["q1"]=test.quantile(q=0.1,axis=1)
test["q2"]=test.quantile(q=0.2,axis=1)
test["q3"]=test.quantile(q=0.3,axis=1)
test["q4"]=test.quantile(q=0.4,axis=1)
test["q5"]=test.quantile(q=0.5,axis=1)
test["q6"]=test.quantile(q=0.6,axis=1)
test["q7"]=test.quantile(q=0.7,axis=1)
test["q8"]=test.quantile(q=0.8,axis=1)
test["q9"]=test.quantile(q=0.9,axis=1)

In [278]:
train

,FLTID,DEPSTN,ARRSTN,STATUS,AC,target,Date_year,Date_month,Date_day,STD_Date_year,...,skew,q1,q2,q3,q4,q5,q6,q7,q8,q9
0,0.009274,0.012510,0.394796,0.868741,0.035323,260.0,2016,1,3,2016,...,1.948756,0.006492,0.026198,0.726558,1.000000,1.922007,3.000000,13.0,96.0,1351.345848
1,0.008949,0.014226,0.394796,0.868741,0.027033,20.0,2016,1,13,2016,...,1.964871,0.006264,0.021910,0.726558,1.000000,1.947301,13.000000,15.1,28.6,1352.384132
2,0.004238,0.394332,0.013373,0.868741,0.035323,0.0,2016,1,16,2016,...,1.964708,0.000000,0.009719,0.286629,1.000000,1.464305,4.400000,16.0,26.6,1352.703690
3,0.001595,0.095073,0.005008,0.868741,0.025836,0.0,2016,1,17,2016,...,1.964842,0.000000,0.000957,0.019588,0.868741,1.000000,3.571874,17.0,18.0,1352.876343
4,0.005230,0.394332,0.014587,0.868741,0.043808,22.0,2016,1,17,2016,...,1.965224,0.003661,0.032120,0.726419,1.000000,1.948202,15.400000,17.5,34.0,1352.096520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,0.028795,0.394332,0.394796,0.122801,0.038226,0.0,2018,7,5,2018,...,1.964545,0.000000,0.000000,0.035396,0.394332,1.945830,5.000000,7.0,22.2,1354.512093
107829,0.008606,0.095073,0.394796,0.122801,0.013363,0.0,2018,1,13,2018,...,1.964530,0.000000,0.000000,0.011936,0.122801,1.000000,1.934518,9.4,14.8,1354.540945
107830,0.002105,0.394332,0.394796,0.122801,0.025836,0.0,2018,11,7,2018,...,1.964713,0.000000,0.001263,0.093712,0.394796,3.482357,7.000000,11.0,27.6,1354.007738
107831,0.007261,0.394332,0.094572,0.868741,0.032541,0.0,2018,1,23,2018,...,1.964926,0.000000,0.004357,0.075963,0.868741,1.000000,5.171941,22.1,27.4,1353.723540


In [279]:
train.columns

Index(['FLTID', 'DEPSTN', 'ARRSTN', 'STATUS', 'AC', 'target', 'Date_year',
       'Date_month', 'Date_day', 'STD_Date_year', 'STD_Date_month',
       'STD_Date_day', 'STD_Date_hour', 'STD_Date_minute', 'STD_Date_second',
       'STA_Date_year', 'STA_Date_month', 'STA_Date_day', 'STA_Date_hour',
       'STA_Date_minute', 'STA_Date_second', 'mean', 'count', 'std', 'min',
       'max', 'kurt', 'skew', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8',
       'q9'],
      dtype='object')

In [280]:
train.drop(['FLTID', 'DEPSTN', 'ARRSTN', 'STATUS', 'AC',  'Date_year',
       'Date_month', 'Date_day', 'STD_Date_year', 'STD_Date_month',
       'STD_Date_day', 'STD_Date_hour', 'STD_Date_minute', 'STD_Date_second',
       'STA_Date_year', 'STA_Date_month', 'STA_Date_day', 'STA_Date_hour',
       'STA_Date_minute', 'STA_Date_second'], axis=1,inplace=True)
test.drop(['FLTID', 'DEPSTN', 'ARRSTN', 'STATUS', 'AC', 'Date_year',
       'Date_month', 'Date_day', 'STD_Date_year', 'STD_Date_month',
       'STD_Date_day', 'STD_Date_hour', 'STD_Date_minute', 'STD_Date_second',
       'STA_Date_year', 'STA_Date_month', 'STA_Date_day', 'STA_Date_hour',
       'STA_Date_minute', 'STA_Date_second'], axis=1,inplace=True)

In [281]:
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# X_train = mms.fit_transform(train_df)
# X_test = mms.transform(test)

In [282]:
# X_train.shape

In [283]:
len(train.columns)

17

In [284]:
train

,target,mean,count,std,min,max,kurt,skew,q1,q2,q3,q4,q5,q6,q7,q8,q9
0,260.0,306.110507,22,686.691696,0.0,2016.0,1.895258,1.948756,0.006492,0.026198,0.726558,1.000000,1.922007,3.000000,13.0,96.0,1351.345848
1,20.0,295.348274,22,688.768264,0.0,2016.0,1.929731,1.964871,0.006264,0.021910,0.726558,1.000000,1.947301,13.000000,15.1,28.6,1352.384132
2,0.0,293.586477,22,689.407381,0.0,2016.0,1.928610,1.964708,0.000000,0.009719,0.286629,1.000000,1.464305,4.400000,16.0,26.6,1352.703690
3,0.0,292.571250,22,689.752687,0.0,2016.0,1.928449,1.964842,0.000000,0.000957,0.019588,0.868741,1.000000,3.571874,17.0,18.0,1352.876343
4,22.0,296.872700,22,688.193040,0.0,2016.0,1.931179,1.965224,0.003661,0.032120,0.726419,1.000000,1.948202,15.400000,17.5,34.0,1352.096520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,0.0,291.284712,22,691.024185,0.0,2018.0,1.927116,1.964545,0.000000,0.000000,0.035396,0.394332,1.945830,5.000000,7.0,22.2,1354.512093
107829,0.0,291.125459,22,691.081890,0.0,2018.0,1.927015,1.964530,0.000000,0.000000,0.011936,0.122801,1.000000,1.934518,9.4,14.8,1354.540945
107830,0.0,294.092375,22,690.015477,0.0,2018.0,1.928708,1.964713,0.000000,0.001263,0.093712,0.394796,3.482357,7.000000,11.0,27.6,1354.007738
107831,0.0,295.637974,22,689.447080,0.0,2018.0,1.929866,1.964926,0.000000,0.004357,0.075963,0.868741,1.000000,5.171941,22.1,27.4,1353.723540


In [285]:
test

,mean,count,std,min,max,kurt,skew,q1,q2,q3,q4,q5,q6,q7,q8,q9
0,307.969613,21,702.959037,0.0,2016.0,1.661115,1.898885,0.005079,0.059745,0.578678,1.803777,4.000000,5.0,6.4,34.0,1490.783615
1,306.964818,21,703.307820,0.0,2016.0,1.660960,1.899031,0.000579,0.019865,0.576813,1.803803,5.000000,5.0,9.0,20.4,1490.923128
2,306.464229,21,703.501452,0.0,2016.0,1.660493,1.898912,0.000000,0.009900,0.164084,1.340322,5.000000,5.6,7.6,22.6,1491.000581
3,308.464776,21,702.750594,0.0,2016.0,1.661905,1.899167,0.002250,0.016993,0.578678,1.804262,5.000000,9.6,11.0,28.6,1490.700238
4,310.150820,21,702.142773,0.0,2016.0,1.662599,1.899135,0.006300,0.039494,0.380928,1.804520,5.000000,10.2,11.4,41.0,1490.457109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,310.365804,21,702.841403,0.0,2018.0,1.663181,1.899443,0.002829,0.029422,0.576813,1.804938,9.000000,12.0,14.4,30.6,1491.936561
9329,309.348912,21,703.236501,0.0,2018.0,1.662212,1.899184,0.000000,0.011443,0.076031,0.389842,1.662212,9.0,21.4,27.4,1492.094600
9330,307.027280,21,704.094219,0.0,2018.0,1.660837,1.899025,0.000064,0.009150,0.235380,1.803750,3.000000,9.0,9.4,20.4,1492.437688
9331,310.782476,21,702.666795,0.0,2018.0,1.663832,1.899669,0.005464,0.053273,0.234137,1.805334,9.000000,14.6,15.0,30.0,1491.866718


In [286]:
len(test.columns)

16

In [294]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
y = mms.fit_transform(train["target"].to_list())

ValueError: Expected 2D array, got 1D array instead:
array=[260.  20.   0. ...   0.   0.   0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [290]:
y

0         260.0
1          20.0
2           0.0
3           0.0
4          22.0
          ...  
107828      0.0
107829      0.0
107830      0.0
107831      0.0
107832      0.0
Name: target, Length: 107833, dtype: float16

In [ ]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

models = []
models.append(LGBMRegressor(random_state=17, max_depth = 3))
models.append(LGBMRegressor(random_state=17,  max_depth = 5))
models.append(XGBRegressor(random_state=17,  max_depth = 3))
models.append(XGBRegressor(random_state=17, max_depth = 5))
models.append(CatBoostRegressor(random_state=17,  max_depth=3))
models.append(CatBoostRegressor(random_state=17,  max_depth=5))

In [ ]:
%%time
for i in range(len(models)):
  models[i].fit(train.drop("target", axis=1),y_train)

In [ ]:
col=train.drop("target", axis=1).columns

In [ ]:
len(col)

In [ ]:
predicts = []
for i in range(len(models)):
  predicts.append(models[i].predict(test[col]))

In [ ]:
pred = np.mean(predicts, axis=0)
true=mms.inverse_transform(pred)

In [ ]:
sample.target = true

In [ ]:
sample

In [ ]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")

In [ ]:
create_submission(sample, "sub5")